# NeuroNexus Analysis Pipeline v6

**S1BF シリコンプローブ記録の統合解析ノートブック**

セルを上から順に実行してください。各ステップは独立して再実行可能です。  
詳しい使い方は `ユーザーガイド.docx` を参照してください。

---

## 0. セットアップ

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

# パイプラインのインポート
from run_analysis import (
    step1_load_data, step2_process_lfp,
    step3_spike_sorting, step3_load_sorting,
    step4_stimulus_analysis, step5_phase_locking,
    step6_save_and_plot, step7_comprehensive,
    step8_explorer, run_full_pipeline,
)

print('Ready!')

## 1. データ読み込み

PLXファイルを選択して読み込みます。ファイルダイアログが開きます。

In [ ]:
from get_path import get_path

# --- ファイル選択 ---
plx_file = get_path('PLXファイルを選択', filetypes=[('PLX', '*.plx')])
print(f'選択: {plx_file}')

# --- 読み込み ---
session = step1_load_data(plx_file)
print(session)

## 2. LFP前処理

バンドパス・ノッチフィルタ → 高調波除去 → Bad ch検出 → ICA（動画あれば）

In [ ]:
lfp_result = step2_process_lfp(session)

print(f"\nLFP shape: {lfp_result['lfp_cleaned'].shape}")
print(f"Good channels: {lfp_result['good_channels']}")
print(f"Noise: {100*np.mean(lfp_result['noise_mask']):.1f}%")

### （任意）LFPを確認

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
t = lfp_result['lfp_times']
lfp = lfp_result['lfp_cleaned']
offset = np.arange(lfp.shape[1]) * 300  # µV offset
for ch in range(lfp.shape[1]):
    ax.plot(t, lfp[:, ch] + offset[ch], lw=0.5)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Channel')
ax.set_title('Cleaned LFP (全チャンネル)')
plt.tight_layout()
plt.show()

## 3. スパイクソーティング

**2つの方法から選んでください：**
- **A) 新規ソーティング** — 自動クラスタリング → キュレーション
- **B) 保存済み読み込み** — 以前の結果（.npz）を再利用

### 3A. 新規ソーティング

`curation` モード:
- `'auto'` — 基準ベースで自動分類（おすすめ）
- `'gui'`  — GUIで手動キュレーション
- `'both'` — 自動 → GUIで確認
- `'none'` — キュレーションなし

In [ ]:
# チャンネルとモードを指定
SORT_CHANNELS = [0]      # 解析するチャンネル（Noneで全チャンネル）
CURATION_MODE = 'auto'   # 'auto' / 'gui' / 'both' / 'none'

sorting_results = step3_spike_sorting(
    session,
    channels=SORT_CHANNELS,
    curation=CURATION_MODE,
)

# 結果の概要
for ch, result in sorting_results.items():
    for u in result.units:
        label = 'Noise' if u.is_noise else ('MUA' if u.is_mua else 'SU')
        print(f'  Ch{ch} Unit{u.unit_id}: {u.n_spikes} spikes, '
              f'SNR={u.snr:.1f}, ISI viol={u.isi_violation_rate:.1%} → {label}')

### 3B. 保存済みソーティングを読み込む

In [ ]:
# ファイルダイアログで .npz を選択
sorting_file = get_path('ソーティング結果を選択', filetypes=[('NPZ', '*.npz')])
sorting_results = step3_load_sorting(sorting_file)

## 4. 刺激応答解析

PSTH・適応解析・条件マスクを計算します。

In [ ]:
protocol, stim_results = step4_stimulus_analysis(
    session, sorting_results, lfp_result
)

### （任意）刺激応答サマリーを表示

In [ ]:
# 最初のユニットの6パネルサマリー
for ch, result in sorting_results.items():
    for unit in result.units:
        if unit.is_noise:
            continue
        print(f'--- Ch{ch} Unit{unit.unit_id} ---')
        protocol.plot_summary(
            unit.spike_times,
            lfp_data=lfp_result['lfp_cleaned'],
            lfp_times=lfp_result['lfp_times'],
            fs=lfp_result['fs']
        )
        plt.show()
        break  # 最初の1ユニットだけ表示
    break

## 5. 位相ロック解析

全ユニット × 全帯域 × 全LFPチャンネル × 全条件の位相ロック解析。

In [ ]:
# 帯域をカスタマイズしたい場合はここで指定（Noneでデフォルト）
FREQ_BANDS = None  # 例: {'theta': (4, 8), 'gamma': (30, 80)}

analyzer = step5_phase_locking(
    sorting_results, lfp_result, protocol,
    freq_bands=FREQ_BANDS,
)

### （任意）ユニットサマリー・集団サマリーを表示

In [ ]:
# 全ユニットの9パネルサマリー
for unit_key, pl in analyzer.unit_results.items():
    analyzer.plot_unit_summary(pl.channel, pl.unit_id)
    plt.show()

In [ ]:
# 集団サマリー
analyzer.plot_population_summary()
plt.show()

## 6. 保存

CSV・NPZ・PNGをまとめて出力します。

In [ ]:
OUTPUT_DIR = os.path.join(os.path.dirname(plx_file), 'output')

step6_save_and_plot(
    analyzer, sorting_results, protocol,
    output_dir=OUTPUT_DIR,
    basename=session.basename,
)
print(f'\n保存先: {OUTPUT_DIR}')

## 7. 全チャンネル統合解析

CSD・深度プロファイル・コヒーレンス行列・グランドサマリー（12パネル）を生成。

In [ ]:
ca = step7_comprehensive(
    session, lfp_result, sorting_results,
    protocol=protocol,
    analyzer=analyzer,
    output_dir=OUTPUT_DIR,
)

## 8. 解析エクスプローラ GUI 🚀

全結果をインタラクティブに探索できる4タブGUIを起動します。

| タブ | 内容 |
|------|------|
| 🧠 スパイク | 品質テーブル・波形一覧・9パネル詳細 |
| 📊 LFP | 全ch波形・PSD・CSD・ヒートマップ |
| 🔗 統合 | 位相ロック深度・STA・条件別MRL |
| 💾 エクスポート | PNG/CSV一括保存 |

In [ ]:
step8_explorer(
    session, lfp_result, sorting_results,
    protocol=protocol,
    sla=analyzer,
    ca=ca,
)

---

## ワンショット実行（全ステップ一括）

上記を個別に実行する代わりに、全ステップをまとめて実行することもできます。

In [ ]:
# === 全ステップ一括実行 ===
# from get_path import get_path
# from run_analysis import run_full_pipeline, launch_explorer_from_results
#
# plx_file = get_path('PLXファイルを選択', filetypes=[('PLX', '*.plx')])
#
# results = run_full_pipeline(
#     plx_file,
#     curation='auto',        # 'auto' / 'gui' / 'both' / 'none'
#     sort_channels=[0],      # Noneで全チャンネル
#     # sorting_file='sorting.npz',  # 保存済みを使う場合
# )
#
# # GUIで探索
# launch_explorer_from_results(results)